In [ ]:
# s3://mlflow-artifacts-capstone-mlops/5/c23904aef6314fb499be6077aad00cb9/artifacts/xgboost_model

In [15]:
import mlflow
import pandas as pd
from mlflow.tracking import MlflowClient
from datetime import datetime
from sklearn.metrics import root_mean_squared_error


In [2]:
# Set MLflow tracking URI
EC2_PUBLIC_DNS = 'ec2-16-16-217-131.eu-north-1.compute.amazonaws.com'
mlflow.set_tracking_uri(f"http://{EC2_PUBLIC_DNS}:5000")


In [3]:
# Specify the run ID
run_id = "c23904aef6314fb499be6077aad00cb9"

# Load model as a PyFuncModel
logged_model = f"runs:/{run_id}/xgboost_model"
loaded_model = mlflow.pyfunc.load_model(logged_model)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[29345.2   25230.064 25230.064 ... 27297.492 31060.291 31060.291]


In [5]:
# Initialize MlflowClient
client = MlflowClient()


In [6]:
# Specify the run ID
run_id = "c23904aef6314fb499be6077aad00cb9"

# Register the model
model_uri = f"runs:/{run_id}/xgboost_model"
model_name = "car-price-regressor-xgboost"
mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'car-price-regressor-xgboost'.
2024/07/27 22:29:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car-price-regressor-xgboost, version 1
Created version '1' of model 'car-price-regressor-xgboost'.


<ModelVersion: aliases=[], creation_timestamp=1722119360050, current_stage='None', description='', last_updated_timestamp=1722119360050, name='car-price-regressor-xgboost', run_id='c23904aef6314fb499be6077aad00cb9', run_link='', source='s3://mlflow-artifacts-capstone-mlops/5/c23904aef6314fb499be6077aad00cb9/artifacts/xgboost_model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [7]:
# Get the latest versions of the registered model
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

/tmp/ipykernel_68361/4173327795.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


version: 1, stage: None


In [8]:
# Transition the model version to 'Staging'
model_version = latest_versions[-1].version
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Staging"
)

/tmp/ipykernel_68361/3922651589.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1722119360050, current_stage='Staging', description='', last_updated_timestamp=1722119371052, name='car-price-regressor-xgboost', run_id='c23904aef6314fb499be6077aad00cb9', run_link='', source='s3://mlflow-artifacts-capstone-mlops/5/c23904aef6314fb499be6077aad00cb9/artifacts/xgboost_model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [9]:
# Update model version description
date = datetime.now().strftime("%Y-%m-%d")
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to Staging on {date}"
)


<ModelVersion: aliases=[], creation_timestamp=1722119360050, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-07-27', last_updated_timestamp=1722119381898, name='car-price-regressor-xgboost', run_id='c23904aef6314fb499be6077aad00cb9', run_link='', source='s3://mlflow-artifacts-capstone-mlops/5/c23904aef6314fb499be6077aad00cb9/artifacts/xgboost_model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [10]:
# Load model as a PyFuncModel
logged_model = f"runs:/{run_id}/xgboost_model"
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [16]:
# Predict on a Pandas DataFrame
data = pd.read_csv('../data/cleaned_car_data.csv')
X = data.drop(columns=['price'])
predictions = loaded_model.predict(X)
print(predictions)

y=data['price']
rmse = root_mean_squared_error(y, predictions)
print(f"RMSE: {rmse}")

[29345.2   25230.064 25230.064 ... 27297.492 31060.291 31060.291]
RMSE: 14554.494803132142


In [17]:
# Transition the model version to 'Production'
model_version = latest_versions[-1].version
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Production"
)


/tmp/ipykernel_68361/4041264306.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1722119360050, current_stage='Production', description='The model version 1 was transitioned to Staging on 2024-07-27', last_updated_timestamp=1722120092542, name='car-price-regressor-xgboost', run_id='c23904aef6314fb499be6077aad00cb9', run_link='', source='s3://mlflow-artifacts-capstone-mlops/5/c23904aef6314fb499be6077aad00cb9/artifacts/xgboost_model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [18]:
# Update model version description
date = datetime.now().strftime("%Y-%m-%d")
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to Production on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1722119360050, current_stage='Production', description='The model version 1 was transitioned to Production on 2024-07-27', last_updated_timestamp=1722120105706, name='car-price-regressor-xgboost', run_id='c23904aef6314fb499be6077aad00cb9', run_link='', source='s3://mlflow-artifacts-capstone-mlops/5/c23904aef6314fb499be6077aad00cb9/artifacts/xgboost_model', status='READY', status_message='', tags={}, user_id='', version='1'>